In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import re, os, json
from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

In [5]:
from typing import TypedDict

#상태 스키마 정의 - 사용자의 선호도, 추천된 메뉴, 메뉴 정보 저장
class MenuState(TypedDict):
    user_preference: str
    recommended_menu: str
    menu_info: str

In [8]:
import random
def get_user_preference(state: MenuState) -> MenuState:
    print("랜덤 사용자 선호도 생성하기...")
    preferences = ["육류", "해산물", "채식", "아무거나"]
    preference = random.choice(preferences)
    print(f"생성 선호: {preference}")
    return {"user_preference":preference}

def recommend_menu(state:MenuState) -> MenuState:
    print("메뉴 추천...")
    preference = state['user_preference']
    menus = {"육류": "스테이크", "해산물":"랍스터 파스타", "채식":"그린 샐러드", "아무거나":"오늘의 쉐프 특선"}
    menu = menus[preference]
    print(f"추천 메뉴: {menu}")
    return {"recommended_menu":menu}

def provided_menu_info(state: MenuState) -> MenuState:
    print("메뉴 정보 제공....")
    menu = state['recommended_menu']
    infos ={
        "스테이크":"최상급 소고기로 만든, 쥬시한 스테이크입니다. 가격: 30,000W", \
        "랍스터 파스타":"신선한 랍스터와 알 단테 파스타의 조화. 가격: 28,000W",\
        "그린 샐러드":"신선한 유기농 채소로 만든 건강한 샐러드. 가격: 15,000W",
        "오늘의 쉐프 특선":"쉐프가 그날그날 엄선한 특별 요리입니다. 가격: 35,000W"
    }
    info = infos[menu]
    print(f"메뉴 정보:{info}")
    return {"menu_info":info}
    
    

In [9]:
from langgraph.graph import StateGraph, START, END

builder= StateGraph(MenuState)

builder.add_node("get_preference", get_user_preference)
builder.add_node("recommend", recommend_menu)
builder.add_node("provide_info", provided_menu_info)

builder.add_edge(START, "get_preference")
builder.add_edge("get_preference", "recommend")
builder.add_edge("recommend", "provide_info")
builder.add_edge("provide_info", END)

graph = builder.compile()

In [12]:
graph

In [16]:
from IPython.display import Image, display
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	get_preference(get_preference)
	recommend(recommend)
	provide_info(provide_info)
	__end__([<p>__end__</p>]):::last
	__start__ --> get_preference;
	get_preference --> recommend;
	provide_info --> __end__;
	recommend --> provide_info;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [18]:
def print_result(result:MenuState):
    print("결과...")
    print(result["user_preference"], result["recommended_menu"], result["menu_info"], sep=', ')

inputs = {"user_preference": ""}
for _ in range(2):
    result = graph.invoke(inputs)
    print_result(result)
    print("*"*100)

랜덤 사용자 선호도 생성하기...
생성 선호: 육류
메뉴 추천...
추천 메뉴: 스테이크
메뉴 정보 제공....
메뉴 정보:최상급 소고기로 만든, 쥬시한 스테이크입니다. 가격: 30,000W
결과...
육류, 스테이크, 최상급 소고기로 만든, 쥬시한 스테이크입니다. 가격: 30,000W
****************************************************************************************************
랜덤 사용자 선호도 생성하기...
생성 선호: 해산물
메뉴 추천...
추천 메뉴: 랍스터 파스타
메뉴 정보 제공....
메뉴 정보:신선한 랍스터와 알 단테 파스타의 조화. 가격: 28,000W
결과...
해산물, 랍스터 파스타, 신선한 랍스터와 알 단테 파스타의 조화. 가격: 28,000W
****************************************************************************************************


In [2]:
from typing import TypedDict, List

class MenuState(TypedDict):
    user_query: str
    is_menu_related: bool
    search_results: List[str]
    final_answer:str
    

In [4]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

embedding_model = OllamaEmbeddings(model="bge-m3:latest")

vector_db = Chroma(
    embedding_function=embedding_model,
    collection_name="restaurant_menu",
    persist_directory="../../chroma_db",
)

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

def get_user_query(state:MenuState) -> MenuState:
    user_query = input("무엇을 도와드릴까요? ")
    return {"user_query" : user_query}

def analyze_input(state:MenuState) -> MenuState:
    analyze_template = """
    사용자의 입력을 분석하여 레스토랑 메뉴 추천이나 음식 정보에 관한 질문인지를 판단하세요.
    사용자 입력:{user_query}
    레스토랑 메뉴나 음식 정보에 관한 질문이면 "True", 아니면 "False"로 답변하세요.
    답변:
    
    """
    analyze_prompt = ChatPromptTemplate.from_template(analyze_template)
    analyze_chain = analyze_prompt | llm | StrOutputParser()
    
    result = analyze_chain.invoke({"user_query": state['user_query']})
    is_menu_related = result.strip().lower() == "true"
    return {"is_menu_related" : is_menu_related}

def search_menu_info(state: MenuState) -> MenuState:
    results = vector_db.similarity_search(state['user_query'], k=2)
    search_results = [doc.page_content for doc in results]
    return {"search_results":search_results}

def generate_menu_response(state: MenuState) -> MenuState:
    response_template = """
    사용자 입력: {user_query}
    메뉴 관련 검색 결과: {search_results}
    
    위 정보를 바탕으로 사용자의 메뉴 관련 질문에 대한 상세한 답변을 생성하세요.
    검색 결과의 정보를 활용하여 정확하고 유용한 정보를 제공하세요.
    
    답변:
    """
    response_prompt = ChatPromptTemplate.from_template(response_template)
    response_chain = response_prompt | llm | StrOutputParser()
    
    final_answer = response_chain.invoke(
        {
            "user_query":state['user_query'],
            "search_results":state["search_results"],
            
        },
    )
    print(f"\n메뉴 어시스턴트: {final_answer}")
    return {"final_answer" : final_answer}

def generate_general_response(state: MenuState) -> MenuState:
    response_template = """
    사용자 입력: {user_query}
    
    위 입력은 레스토랑 메뉴나 음식과 관련이 없습니다.
    일반적인 대화 맥락에서 적절한 답변을 생성하세요.
    
    답변:
    """
    response_prompt = ChatPromptTemplate.from_template(response_template)
    response_chain = response_prompt | llm | StrOutputParser()
    final_answer = response_chain.invoke({"user_query": state["user_query"]})
    print(f"\n일반 어시스턴트:{final_answer}")
    return {"final_answer":final_answer}     


In [8]:
from typing import Literal

def decide_next_step(state : MenuState) -> Literal["search_menu_info", "generate_general_response"]:
    if state['is_menu_related']:
        return "search_menu_info"
    else:
        return "generate_general_response"

In [ ]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(MenuState)

builder.add_node("get_user_query", get_user_query)
builder.add_node("analyze_input", analyze_input)
builder.add_node("search_menu_info", search_menu_info)
builder.add_node("generate_menu_response", generate_menu_response)
builder.add_node("generate_general_response", generate_general_response)

builder.add_edge(START, "get_user_query")
builder.add_edge("get_user_query", "analyze_input")

builder.add_conditional_edges(
    "analyze_input", 
    decide_next_step, 
    {
        "search_menu_info" : "search_menu_info",
        "generate_general_response" : "generate_general_response"
    }
)

builder.add_edge("search_menu_info", "generate_menu_response")
builder.add_edge("generate_menu_response", END)
builder.add_edge("generate_general_response", END)

graph = builder.compile()

In [15]:
from IPython.display import Image, display
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	get_user_query(get_user_query)
	analyze_input(analyze_input)
	search_menu_info(search_menu_info)
	generate_menu_response(generate_menu_response)
	generate_general_response(generate_general_response)
	__end__([<p>__end__</p>]):::last
	__start__ --> get_user_query;
	generate_general_response --> __end__;
	generate_menu_response --> __end__;
	get_user_query --> analyze_input;
	search_menu_info --> generate_menu_response;
	analyze_input -.-> search_menu_info;
	analyze_input -.-> generate_general_response;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [22]:
while True:
    initial_state = {'user_query':''}
    graph.invoke(initial_state)
    continue_chat = input("다른 질문이 있으신가요? (y / n): ").lower()
    if continue_chat != 'y':
        break


메뉴 어시스턴트: 스테이크 메뉴의 가격은 다음과 같습니다:

1. **시그니처 스테이크** - ₩35,000  
   - **주요 식재료**: 최상급 한우 등심, 로즈메리 감자, 그릴드 아스파라거스  
   - **설명**: 셰프의 특제 시그니처 메뉴로, 21일간 건조 숙성한 최상급 한우 등심을 사용합니다. 미디엄 레어로 조리하여 육즙을 최대한 보존하며, 로즈메리 향의 감자와 아삭한 그릴드 아스파라거스가 곁들여집니다. 레드와인 소스와 함께 제공되어 풍부한 맛을 더합니다.

2. **안심 스테이크 샐러드** - ₩26,000  
   - **주요 식재료**: 소고기 안심, 루꼴라, 체리 토마토, 발사믹 글레이즈  
   - **설명**: 부드러운 안심 스테이크를 얇게 슬라이스하여 신선한 루꼴라 위에 올린 메인 요리 샐러드입니다. 체리 토마토와 파마산 치즈 플레이크로 풍미를 더하고, 발사믹 글레이즈로 마무리하여 고기의 풍미를 한층 끌어올렸습니다.

이 두 가지 스테이크 메뉴가 있으며, 각각의 가격과 특징을 참고하여 선택하시면 좋겠습니다.

일반 어시스턴트:미국의 수도는 워싱턴 D.C.입니다.
